In [1]:
# Login to fb @news page
def login(driver, account, password): 
    
    # Enter account
    driver.find_element_by_xpath("//*[@id=\"m_login_email\"]").send_keys(account)

    # Enter password
    driver.find_element_by_xpath("//*[@id=\"m_login_password\"]").send_keys(password)

    # Submit login
    driver.find_element_by_xpath("//*[@id=\"u_0_5\"]").click()
    time.sleep(2)
    
    # Proceed to home page
    driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div/div[3]/div[1]/div/div/a').click()
    time.sleep(2)

# Get to article comment section
def toCommentSection(page_id, driver, article_id):
    # Generate url to each article comment section
    url_article = 'https://mobile.facebook.com/story.php?story_fbid=' + article_id + '&id=' + page_id + '&fs=5&focus_composer=0&ref=page_internal'
    driver.get(url_article)

def initCSV(page):
    import csv
    with open('results/'+page+'_posts.csv', 'w', encoding="utf-8") as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['article_index', 'headline', 'status', 'has_headline', 
                             'has_status', 'has_img', 'has_vid', 'num_hashtags', 'hashtag_text', 'likes'])
        
def appendResults(page, posts):
    df = pd.DataFrame(np.array(posts), columns=['article_index', 'headline', 'status', 'has_headline', 
                                                'has_status', 'has_img', 'has_vid', 'num_hashtags', 'hashtag_text', 'likes'])
    
    # Append results to existing file
    with open('results/'+page+'_posts.csv', 'a', encoding="utf-8") as f:
        df.to_csv(f, header=False, index=False)

def getStatusLikes(driver, article_index):
    headline = ''
    status = ''
    has_headline = 0
    has_vid = 0
    has_img = 0
    has_status = 0
    num_hashtags = 0
    hashtag_text = []
    likes = '0'
    
    try: 
        headline = driver.find_element_by_css_selector("span[data-ad-preview=\"headline\"]").text
        has_headline = 1
    except:
        headline = ''
        try:
            driver.find_element_by_css_selector("i[data-ad-preview=\"video-cover\"]")
            has_vid = 1
        except:
            pass
        try:
            driver.find_element_by_css_selector("i[data-sigil=\"photo-image\"]")
            has_img = 1
        except:
            pass
    
    try:
        if has_img == 0:
            status = driver.find_element_by_css_selector("div[data-ad-preview=\"message\"]").text
            has_status = 1
        else:
            status = driver.find_element_by_css_selector("div[class=\"msg\"]").text
            has_status = 1
    except:
        pass
    
    try:
        likes = driver.find_element_by_css_selector("div[class=\"_1g06\"]").text
    except:
        pass
    
    try:
        hashtags = driver.find_elements_by_css_selector("span[class=\"_5ayu\"]")
        num_hashtags = len(hashtags)
        hashtag_text = [hashtag.text for hashtag in hashtags]
    except:
        pass
    
    hashtag_text = ','.join(hashtag_text) 
    res_tuple = (article_index, headline, status, has_headline, has_status, has_img, has_vid, num_hashtags, hashtag_text, likes)
    return res_tuple

In [2]:
from selenium import webdriver
import pandas as pd
import numpy as np
import time
# Start webdriver
driver = webdriver.Chrome()
driver.get(account)

login(driver, password)

pages = ['ettoday', 'apple', 'yahoo', 'udn', 'set', 'ltn']
for page in pages:
    posts = []
    initCSV(page)
    article_ids = pd.read_csv('/Users/Shayne/jupyter/web-scraping/fb-data/parallelise-all/results-1000-win-fixed/'+page+'_ids.csv', header=None)[0]
    page_ids = pd.read_csv('page_ids.csv', header=0, index_col=0, dtype=str)
    page_id = page_ids.loc[page].get('page_id')
    to_redo = pd.read_csv('redo_index_'+page+'.csv', header=None, index_col=0)[1]
    for article_index in to_redo:
        article_id = article_ids[article_index]
        toCommentSection(page_id, driver, article_id)
        posts.append(getStatusLikes(driver, article_index))
    appendResults(page, posts)
